In [2]:
!gdown https://drive.google.com/uc?id=13GI1F1hvwpMUGBSa0QC6ov4eE57GC_Zx

Downloading...
From: https://drive.google.com/uc?id=13GI1F1hvwpMUGBSa0QC6ov4eE57GC_Zx
To: /home/zaid/evals/evals/datasets/UNv1.0.testsets.tar.gz
100%|██████████████████████████████████████| 3.62M/3.62M [00:01<00:00, 2.68MB/s]


In [3]:
!tar -xzvf UNv1.0.testsets.tar.gz

testsets/
testsets/UNv1.0.pdf
testsets/testset/
testsets/testset/UNv1.0.testset.fr
testsets/testset/UNv1.0.testset.ids
testsets/testset/UNv1.0.testset.ar
testsets/testset/UNv1.0.testset.zh
testsets/testset/UNv1.0.testset.ru
testsets/testset/UNv1.0.testset.en
testsets/testset/UNv1.0.testset.es
testsets/devset/
testsets/devset/UNv1.0.devset.fr
testsets/devset/UNv1.0.devset.es
testsets/devset/UNv1.0.devset.ru
testsets/devset/UNv1.0.devset.ids
testsets/devset/UNv1.0.devset.zh
testsets/devset/UNv1.0.devset.en
testsets/devset/UNv1.0.devset.ar
testsets/README
testsets/DISCLAIMER


In [4]:
src = "ar"
trg = "en"

In [5]:
sys_msg = f"Translate the following statement from {src} to {trg}"
def create_chat_prompt(sys_msg, input_text):
    return [
        {"role": "system", "content": sys_msg}, 
        {"role": "user", "content": input_text}
    ]

def create_chat_example(article, summary):
    return [
        {"role": "system", "content": article, "name": "example_user"},
        {"role": "system", "content": summary, "name": "example_assistant"},
    ]

In [6]:
!mkdir -p ../registry/data/unv1

In [7]:
import pandas as pd

src_path = "testsets/testset/UNv1.0.testset.ar"
trg_path = "testsets/testset/UNv1.0.testset.en"

df_test_tar = pd.read_csv(trg_path, delimiter="  ", header = None)
df_test_src = pd.read_csv(src_path, delimiter="  ", header = None)

df_test = pd.concat([df_test_tar, df_test_src], axis = 1)
df_test.columns = ['target', "source"]

src_path = "testsets/devset/UNv1.0.devset.ar"
trg_path = "testsets/devset/UNv1.0.devset.en"

df_dev_tar = pd.read_csv(trg_path, delimiter="  ", header = None)
df_dev_src = pd.read_csv(src_path, delimiter="  ", header = None)

df_dev = pd.concat([df_dev_tar, df_dev_src], axis = 1)
df_dev.columns = ['target', "source"]

/tmp/ipykernel_753/2521196268.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_test_tar = pd.read_csv(trg_path, delimiter="  ", header = None)
/tmp/ipykernel_753/2521196268.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_test_src = pd.read_csv(src_path, delimiter="  ", header = None)
/tmp/ipykernel_753/2521196268.py:15: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_dev_tar = pd.read_csv(trg_path,

In [8]:
df_test

,target,source
0,"7439th meeting, held on 11 May 2015.",الجلسة 7439 المعقودة في 11 أيار/مايو ٢٠١٥.
1,ISIL itself has published videos depicting peo...,وقد نشر التنظيم نفسه أشرطة فيديو تبيّن تعرض أش...
2,UNICEF disbursed emergency cash assistance to ...,وصرفت منظمة اليونيسيف مساعدة نقدية طارئة لعشرا...
3,31. Recognizes the important contribution of t...,31 - تقر بأن الآلية الأفريقية لاستعراض الأقران...
4,Spreads between sovereign bonds in Germany and...,وبقي هامش أسعار الفائدة بين السندات السيادية ف...
...,...,...
3995,1. Declare our absolute support for the legiti...,١ - تأييدنا المطلق للشرعية الدستورية ممثلة بفخ...
3996,The States of the zone have adopted a plan of ...,وقد اعتمدت دول المنطقة خطة عمل لتعزيز الأمن ال...
3997,The States parties to the Treaty unequivocally...,وقد تعهدت الدول الأطراف في المعاهدة بصورة قطعي...
3998,136th meeting,الاجتماع 136


In [9]:
df_dev["sample"] = df_dev.apply(lambda x: create_chat_example(x['source'], x['target']), axis=1)
df_dev[["sample"]].to_json(f'../registry/data/unv1/few_shot_{src}_{trg}.jsonl', lines=True, orient="records")

df_test["input"] = df_test['source'].apply(lambda x: create_chat_prompt(sys_msg, x))
df_test["ideal"] = df_test['target']
df_test[["input", "ideal"]].to_json(f'../registry/data/unv1/samples_{src}_{trg}.jsonl', lines=True, orient="records")

In [10]:
pos_tagging_task_specs = """
unv1:
    id: unv1.test.v1
    metrics: [accuracy]
    description: Evaluate Arabic translation
# Define the eval
unv1.test.v1:
  # Specify the class name as a dotted path to the module and class
  class: evals.elsuite.translate:Translate
  args:
    samples_jsonl: unv1/samples_ar_en.jsonl
    num_few_shot: 0 # max few shots to use

""".strip()
with open("../registry/evals/unv1.yaml", "w") as file:
    file.write(pos_tagging_task_specs)


In [12]:
import os
os.environ["OPENAI_API_KEY"] = "<openai-key>"
os.environ["EVALS_THREADS"] = "3"
os.environ["EVALS_THREAD_TIMEOUT"] = "600"

In [13]:
record_path = "../eval_results/unv1.jsonl"
!oaieval gpt-3.5-turbo-0301 unv1 --record_path {record_path} --seed 41 --modelspec_extra_options temperature=0.0 --max_samples 4000

[2023-05-17 00:39:13,256] [registry.py:156] Loading registry from /home/zaid/evals/evals/registry/evals
[2023-05-17 00:39:13,492] [registry.py:156] Loading registry from /home/zaid/.evals/evals
[2023-05-17 00:39:14,335] [oaieval.py:213] Run started: 230516213914SKD332HA
[2023-05-17 00:39:14,377] [data.py:75] Fetching unv1/samples_ar_en.jsonl
[2023-05-17 00:39:14,482] [eval.py:32] Evaluating 4000 samples
[2023-05-17 00:39:14,492] [eval.py:152] Running in threaded mode with 3 threads!
  1%|▍                                      | 40/4000 [00:53<1:37:47,  1.48s/it][2023-05-17 00:40:08,558] [_common.py:105] Backing off openai_chat_completion_create_retrying(...) for 0.5s (openai.error.RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-6VIsbC1WgU4rx2bWplxNV7gP on tokens per min. Limit: 90000 / min. Current: 86576 / min. Contact us through our help center at help.openai.com if you continue to have issues.)
[2023-05-17 00:40:09,699] [_common.py:105] Backing off o

In [ ]:
import pandas as pd

with open(record_path, "r") as f:
    events_df = pd.read_json(f, lines=True)
print(events_df[events_df["final_report"].notnull()]["final_report"].to_list())

In [ ]:
from sacrebleu.metrics.bleu import BLEU

for i, r in pd.json_normalize(events_df[events_df.type == "sampling"].sort_values('sample_id').data).iterrows():
    print(f"Prompt: {r.prompt[-1]}")
    print(f"Sampled: {r.sampled}")
    print(f"Truth: {df_test['target'][i]}")
    print("score", BLEU(effective_order = True).sentence_score(r.sampled, [df_test['target'][i]]).score)
    print(f"{i}","--" * 25)
    break

In [ ]:
events_df[events_df.type == "sampling"].sort_values('sample_id')['sample_id']